Universidad del Valle de Guatemala  
Aprendizaje por Refuerzo
Alberto Suriano  

Laboratorio 4  
Marlon Hernández - 15177  

- Link del repositorio: https://github.com/ivanhez/RL-LAB4

## Task 1
Responda a cada de las siguientes preguntas de forma clara y lo más completamente posible.

1. **¿Cómo afecta la elección de la estrategia de exploración (exploring starts vs soft policy) a la precisión de la evaluación de políticas en los métodos de Monte Carlo?**
    
- a. Considere la posibilidad de comparar el desempeño de las políticas evaluadas con y sin explorar los inicios o con diferentes niveles de exploración en políticas blandas.

    El Exploring Starts es una técnica que asegura que todas las posibles acciones desde cada estado se exploren al iniciar los episodios desde cada posible estado-acción. Esto permite que la política se evalúe de manera completa en el espacio de estados, esto lleva a una evaluación más precisa de la política. Sin embargo, esta estrategia puede ser poco práctica en entornos más complejos o cuando el espacio de estados es muy grande.
    
    En las Soft Policy todas las acciones tienen una probabilidad mayor que cero de ser seleccionadas y permiten una exploración continua sin necesidad de iniciar en cada estado-acción. Aunque esta estrategia facilita la implementación y es más flexible, puede llevar más tiempo converger a una evaluación precisa de la política debido a que algunas áreas del espacio de estados pueden llegar a ser no exploradas.

2. **En el contexto del aprendizaje de Monte Carlo fuera de la póliza, ¿cómo afecta la razón de muestreo de importancia a la convergencia de la evaluación de políticas? Explore cómo la razón de muestreo de importancia afecta la estabilidad y la convergencia**  
    La razón de muestreo de importancia es crítica en métodos de Monte Carlo off-policy, ya que ajusta las recompensas obtenidas bajo una política de comportamiento para estimar los valores bajo una política objetivo. Una mala elección en la política de comportamiento puede llevar a razones de muestreo de importancia amplias, lo que afecta la varianza y las estimaciones. Si la política de comportamiento es significativamente diferente de la política objetivo, las razones de muestreo de importancia pueden variar ampliamente, aumentando la varianza y afectando negativamente la convergencia.
    
3. **¿Cómo puede el uso de una soft policy influir en la eficacia del aprendizaje de políticas óptimas en comparación con las políticas deterministas en los métodos de Monte Carlo? Compare el desempeño y los resultados de aprendizaje de las políticas derivadas de estrategias épsilon-greedy con las derivadas de políticas deterministas**  
    Las políticas derivadas de estrategias épsilon-greedy pueden ofrecer un mejor balance entre exploración y explotación, adaptándose mejor a cambios en el entorno y aprendiendo políticas óptimas de manera más efectiva en entornos complejos.
    
    Las soft policies permiten una exploración continua del espacio de estados, lo que puede ser beneficioso en entornos dinámicos o desconocidos. Esto puede conducir a un aprendizaje más robusto pero más lento, mientras que las políticas deterministas: Pueden converger más rápidamente en entornos estables donde la relación entre acciones y recompensas es bien entendida. Sin embargo, carecen de la capacidad de explorar alternativas que podrían resultar en descubrimientos de estrategias más eficientes.

4. **¿Cuáles son los posibles beneficios y desventajas de utilizar métodos de Monte Carlo off-policy en comparación con los on-policy en términos de eficiencia de la muestra, costo computacional. y velocidad de aprendizaje?**
    Las políticas Off-Policy permiten reutilizar experiencias pasadas acumuladas bajo diferentes políticas de comportamiento, lo que aumenta la eficiencia de las muestras. Sin embargo, requiere métodos adicionales para ajustar las diferencias entre las políticas de comportamiento y objetivo, lo que puede incrementar el costo computacional y afectar la velocidad de aprendizaje debido a problemas de varianza alta. Con las políticas On-Policy se aprende directamente basándose en las acciones tomadas, lo que puede resultar en una convergencia más rápida y estable.

## Task 2
En este ejercicio, simulará un sistema de gestión de inventarios para una pequeña tienda minorista. La tienda tiene como objetivo maximizar las ganancias manteniendo niveles óptimos de existencias de diferentes productos.
Utilizará métodos de Monte Carlo para la evaluación de pólizas, exploring starts, soft policies y aprendizaje off-policy para estimar el valor de diferentes estrategias de gestión de inventarios. Su objetivo es implementar una solución en Python y responder preguntas específicas en función de los resultados.
Instrucciones:
1. Defina el entorno:

- a. Utilice el ambiente dado más adelante para simular el entorno de la tienda. Considere que:
  - i. El estado representa los niveles de existencias actuales de los productos.
  - ii. Las acciones representan decisiones sobre cuánto reponer de cada producto.

2. Generar episodios:

- a. Cada episodio representa una serie de días en los que la tienda sigue una política de inventario específica.
- b. Debe recopilar datos para varios episodios y registrar las recompensas (ganancias) de cada día.

3. Exploring Starts:

- a. Implemente explorar inicios para garantizar un conjunto diverso de estados y acciones iniciales.

4. Soft Policies:

- a. Utilice una soft policy (como epsilon-greedy) para garantizar un equilibrio entre la exploración y la explotación.

5. Aprendizaje off-policy:
        
- a. Implemente el aprendizaje off-policy para evaluar una política objetivo utilizando datos generados por una política de comportamiento diferente.

In [20]:
import numpy as np
import random

# definicion del ambiente
class InventoryEnvironment:
    def __init__(self):
        self.products = ['product_A', 'product_B']
        self.max_stock = 10
        self.demand = {'product_A': [0, 1, 2], 'product_B': [0, 1, 2]}
        self.restock_cost = {'product_A': 5, 'product_B': 7}
        self.sell_price = {'product_A': 10, 'product_B': 15}
        self.state = None

    def reset(self):
        self.state = {product: random.randint(0, self.max_stock) for product in self.products}
        return self.state

    def step(self, action):
        reward = 0
        for product in self.products:
            stock = self.state[product]
            restock = action[product]
            self.state[product] = min(self.max_stock, stock + restock)
            demand = random.choice(self.demand[product])
            sales = min(demand, self.state[product])
            self.state[product] -= sales
            reward += sales * self.sell_price[product] - restock * self.restock_cost[product]
        return self.state, reward

# init del ambiente
env = InventoryEnvironment()

# politica epsilon greedy
def epsilon_greedy_policy(state, epsilon=0.1):
    if random.uniform(0, 1) < epsilon:
        return {product: random.randint(0, env.max_stock) for product in env.products}
    else:
        return {product: env.max_stock - state[product] for product in env.products}

# generar un episodio
def generate_episode(policy, exploring_starts=False):
    state = env.reset()
    if exploring_starts:
        action = {product: random.randint(0, env.max_stock) for product in env.products}
    else:
        action = policy(state)
    states, actions, rewards = [state], [action], []
    for _ in range(10):
        next_state, reward = env.step(action)
        action = policy(next_state)
        states.append(next_state)
        actions.append(action)
        rewards.append(reward)
    return states, actions, rewards

# evaluar la politica usando Monte Carlo off-policy
def off_policy_evaluation(policy, behavior_policy, episodes=1000, gamma=0.9):
    G = 0
    C = 0
    for _ in range(episodes):
        states, actions, rewards = generate_episode(behavior_policy)
        W = 1.0
        for t in range(len(states)):
            G = gamma * G + rewards[t]
            if actions[t] != policy(states[t]):
                break
            W /= 1.0 / (1.0 / len(env.products))
            C += W
    return G / C if C != 0 else 0

# hiperparametros
epsilon = 0.1
gamma = 0.9
episodes = 1000

# politicas
target_policy = lambda state: {product: env.max_stock - state[product] for product in env.products}
behavior_policy = lambda state: epsilon_greedy_policy(state, epsilon)

# evaluar la politica objetivo usando datos generados por la politica de comportamiento
value_estimate = off_policy_evaluation(target_policy, behavior_policy, episodes, gamma)
print(f"Valor estimado de la política objetivo: {value_estimate}")


Valor estimado de la política objetivo: -87.46537553519286


Preguntas para responder:
1. ¿Cuál es el valor estimado de mantener diferentes niveles de existencias para cada producto?

In [21]:
def value_estimation_per_stock_level():
    value_estimates = {}
    for stock_A in range(env.max_stock + 1):
        for stock_B in range(env.max_stock + 1):
            state = {'product_A': stock_A, 'product_B': stock_B}
            value_estimates[(stock_A, stock_B)] = off_policy_evaluation(target_policy, behavior_policy, episodes, gamma)
    return value_estimates

stock_values = value_estimation_per_stock_level()
print(f"Valor estimado por niveles de existencias: {stock_values}")

res = 0
for val in stock_values.values(): 
    res += val 

res = res / len(stock_values) 
  
print(f"Valor estimado promedio: {res}") 

Valor estimado por niveles de existencias: {(0, 0): -83.73780709644494, (0, 1): -74.07435426290122, (0, 2): -140.61648845660397, (0, 3): -80.62832635842103, (0, 4): -187.60160938560406, (0, 5): -50.877270861632454, (0, 6): -56.13373757227087, (0, 7): -110.35914871796778, (0, 8): -113.4442146912103, (0, 9): -279.74006024258966, (0, 10): -97.39195434832573, (1, 0): -49.20110472549334, (1, 1): -85.91347659102001, (1, 2): -201.78923434712362, (1, 3): -88.00687524641862, (1, 4): -65.28551222641815, (1, 5): -96.46419691615556, (1, 6): -121.47518560501533, (1, 7): -93.55821708434306, (1, 8): -79.32127152868073, (1, 9): -111.03819314350116, (1, 10): -58.545444579573974, (2, 0): -135.9018823139432, (2, 1): -161.3022981193048, (2, 2): -83.0896135535379, (2, 3): -65.29983096334546, (2, 4): -35.492793038562795, (2, 5): -109.3354346716919, (2, 6): -63.72399907566641, (2, 7): -62.643854011921405, (2, 8): -153.000946915906, (2, 9): -40.353501050278666, (2, 10): -115.37148862606645, (3, 0): -70.165430

2. ¿Cómo afecta el valor epsilon en la política blanda al rendimiento?

In [22]:
epsilon_values = [0.01, 0.1, 0.2, 0.5]
performance = {}
for eps in epsilon_values:
    behavior_policy = lambda state: epsilon_greedy_policy(state, eps)
    performance[eps] = off_policy_evaluation(target_policy, behavior_policy, episodes, gamma)
print(f"Rendimiento para diferentes valores de epsilon: {performance}")

Rendimiento para diferentes valores de epsilon: {0.01: -54.435769377566544, 0.1: -57.744221662259335, 0.2: -81.2231140523781, 0.5: -120.52548268795776}


3. ¿Cuál es el impacto de utilizar el aprendizaje fuera de la política en comparación con el aprendizaje dentro de la política

In [23]:
on_policy_value = off_policy_evaluation(target_policy, target_policy, episodes, gamma)
off_policy_value = off_policy_evaluation(target_policy, behavior_policy, episodes, gamma)
print(f"Valor on-policy: {on_policy_value}")
print(f"Valor off-policy: {off_policy_value}")

Valor on-policy: -102.0975479664497
Valor off-policy: -56.80675675601024


Referencias:
- https://medium.com/nerd-for-tech/monte-carlo-methods-for-reinforcement-learning-d30d874dd817
- https://medium.com/@hsinhungw/intro-to-reinforcement-learning-monte-carlo-to-policy-gradient-1c7ede4eed6e
- https://www.geeksforgeeks.org/monte-carlo-policy-evaluation/ 
- https://cwkx.github.io/data/teaching/dl-and-rl/rl-lecture5.pdf
- https://bechirtr97.medium.com/on-policy-vs-off-policy-monte-carlo-control-methods-for-supply-chain-optimization-a-use-case-of-7b8d0d2e80e6